## In-class Lesson 2: Pivots, Joins and Functions in `pandas`

Building off last week's lesson, we'll be working with some election results data today as we learn how to use the `pivot_table` and `merge` methods in `pandas`.

We'll be working with two csvs today. The first called `ks_ballot.csv` is a file of county-level results from the Kansas Constitutional Amendment on abortion access. The second called `ks_prez.csv` is a file containing county-level presidential results in Kansas from the 2020 election.

Our goal: Join the two csvs together into a single dataframe and compare the 2020 presidential results to the results of the ballot initiative.

## Import `pandas`

In [ ]:
import pandas as pd

## Load `ks_ballot.csv` from your computer

`pandas` locates files using their location on your computer, known as the path. Paths look like this for example:

```/Users/user/Downloads/file.csv```

Below we can use the `./data/` because the files are in the `data` folder with our notebook.

In [ ]:
ks_ballot = pd.read_csv('./data/ks_ballot.csv')

ks_ballot.head()

## Inspecting our data

You'll notice a few familiar things about this data. One, it contains a county level `fips` code. But notice that it is different than other county-level data we've worked with. Getting a unique count of the `fips` column using pandas `value_counts()` method will illustrate the difference.

In [ ]:
ks_ballot['fips'].value_counts().head()

As we see, each county fips is repeated twice because there are two answers on the ballot measure: Yes or No. This data is stored in what is called `long format` where each row is a separate entry for each ballot choice. Our goal is to join this data to the presidential results data, so let's take a look at it.

## Load `ks_prez.csv` from our computer

In [ ]:
ks_prez = pd.read_csv('./data/ks_prez.csv')

ks_prez.head()

Notice here our data is stored differently. Not only do we have additional details about the counties (name, state and a value called `acp_type` but it also appears to be stored in what is called `wide format` where each county has it's own row. To be sure, let's use `value_counts()` again on the `fips` column.


In [ ]:
ks_prez['fips'].value_counts().head()

As expected, each fips appears only once in the presidential election data. Since our goal is to join these two dataframes together, we are going to need to convert them to the same format. For this exercise, we'll take the long formatted ballot initiative results and turn them into wide format, which will make for an easier comparison of the two elections in each county. To do this, we will use a tool we've used before: a PivotTable. 

In `pandas`, creating a PivotTable is very similar to how we did it in Excel or GoogleSheets. We choose the rows, or `index` in pandas, the columns and the values to aggregate, usually a `count` or `sum`. 

Returning to `ks_ballot`, our index will be the `fips` code because we want to group our data into a table that looks something like this:

| fips  | yes  | no   |
|-------|------|------|
| 20001 | 1836 | 1816 |
| 20003 | 1418 | 950  |

## Working with fips and other numbers stored as text

Because we're working with `fips` codes, I want to pause here and reimport our data passing an argument that will save us some headaches later. You'll probably recall that some `fips` codes have a leading zero, such as `05001`, which can be cut off if the code is read is as a number, rather than as text. We don't have that issue with Kansas because it's fips code is `20` but that may not be the case every time. To make sure you don't lose that zero, we'll use the `dtype` argument in the `read_csv` method to set the `fips` column as text, or `object` in `pandas`, right from the start.  

In [ ]:
ks_prez = pd.read_csv('./data/ks_prez.csv', dtype={'fips':'object'})

ks_ballot = pd.read_csv('./data/ks_ballot.csv', dtype={'fips':'object'})

## Inspecting datatypes

Let's take a look at the datatypes for each dataframe using the `dtypes` method in `pandas`. And we'll see the `fips` column in each is now stored as an object.

In [ ]:
ks_ballot.dtypes

In [ ]:
ks_prez.dtypes

## Creating a `pivot_table` in pandas

There are a few different ways to pivot and group data in `pandas`. We'll be using the `pivot_table` method in this example. You can read the full documentation [here](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

First, let's look at the columns we have in our dataframe. You can returns these as a list like below.

## List columns in a dataframe

In [ ]:
ks_ballot.columns.to_list()

Let's look at the code below. First, we invoke the `pivot_table` method from pandas, which we've assigned the variable name `pd`. Next, we'll pass lists to the method using our column names. 

We'll be summing vote totals, so `votes` goes in values.

We want to group by the `fips`, so we'll pass `fips` to the index argument.

We want to break out our `choice` column into two seperate columns becase on the unique values in the columns. So we pass it to the columns argument.

Lastly, I'm going to do two things to our pivot_table. I'm going to reset the index so that `fips` becomes a true column in our destination dataframe. I'm also going to fill any null values with 0's becauses we'll be doing math on this data.

Let's try it out, assigning the pivot table as a variable named `ks_wide`

In [ ]:
ks_wide = pd.pivot_table(ks_ballot, 
               values= ['votes'], 
               index = ['fips'], 
               columns=['choice'], 
               aggfunc=sum).reset_index().fillna(0)

ks_wide.head()

This looks like what we want with the exception of the names of the columns. We have created a dataframe that has two layers of column headers when we really only want one layer. We can fix that by renaming the columns to names of our choice using a list.

## Rename columns

In [ ]:
ks_wide.columns = ['fips', 'no', 'yes']

ks_wide.head()

Now we have what we want. A single row per county with no votes and yes votes in their own columns. Now, let's do a little math. 

Because we're going to be comparing this to vote percentages, let's calculate a total of votes in one column and then calculate the vote shares for yes and no. Let's also round those vote shares to three decimal places and multiply by 100 to represent them as percentages. We can do this pretty easily in pandas like below.

In [ ]:
ks_wide['total'] = ks_wide['no'] + ks_wide['yes']

ks_wide['yes_pct'] = round(ks_wide['yes'] / ks_wide['total'], 3) * 100

ks_wide['no_pct'] = round(ks_wide['no'] / ks_wide['total'], 3) * 100

ks_wide.head()

Let's also add percentages to our presidential results data so we can compare the vote shares later when we join the tables together.

In [ ]:
ks_prez['biden_pct'] = round(ks_prez['biden'] / ks_prez['prez_total'], 3) * 100

ks_prez['trump_pct'] = round(ks_prez['trump'] / ks_prez['prez_total'], 3) * 100

## Joining dataframes

Now that we have each dataframe formated as we'd like. We're ready to join them together. In `pandas`, there are a few ways we can do this but today we'll be using the `merge` method.

Just like joining in SQL, we will use a column containig common values in each data frame. In this case, that's the `fips` column in each dataframe. We know these dataframes each contain a record per county so we'll be using an `inner` join to connect records that appear in both dataframes. 

Also like SQL, `merge` uses a left/right frame of reference. So the first table we pass in is the left table and the second table is the right table. In our example, it doesn't really matter which one is which, particularly since we can always rearrange the columns later.

Below we pass in `ks_prez` and `ks_wide` and then assign the joined dataframe to the variable `ks`.

In [ ]:
ks = pd.merge(ks_prez, ks_wide, how='inner', on='fips')

ks

Congrats! You've joined two dataframes in `pandas`. We now have one table with the presidential results and the ballot initiative choices.

We are ready to do some analysis. Let's start with just a few questions:

## What were the statewide totals for `Yes` and `No`?

We can get this by using pandas `sum` method.

In [ ]:
ks[['yes','no','total']].sum()

## How many counties did `No` win? Which ones?

Because there are only two options, Yes or No, we can answer this by looking purely at the vote share percentages above and below 50 percent. As we've discussed before, when there are more candidates, we need to use a different method finding the largest vote getter of a group to assign a winner (candidates can and often do win races with less than 50% of the vote). 

We can do this with a filter and using `1en()` to get the count.

In [ ]:
no_cos = ks[ks['no_pct'] > 50]

no_cos

In [ ]:
len(no_cos)

## How many `No` counties were also Trump counties in 2020? Which ones?

This adds another level of complexity to our analysis.

In [ ]:
trump_nos = no_cos[no_cos['trump_pct'] > no_cos['biden_pct']]

trump_nos

In [ ]:
len(trump_nos)

## In how many, if any counties, did `yes` get a higher vote share than former President Donald Trump? Which counties?

In [ ]:
ks[ks['yes_pct'] > ks['trump_pct']]

You can add even more questions and analysis from here if you like. Hopefully this has helped you start to feel comfortable with some of the ways `pandas` can help with your data analysis.

## Lastly, let's save `ks` to a csv in case we want to use it elsewhere.

In [ ]:
ks.to_csv('./data/ks.csv', index=False)